# [DataFountain:阿尔茨海默症的识别](https://www.datafountain.cn/competitions/369/datasets)

## 数据特点

- 训练数据集，数据格式为h5,主键为data。该数据集共有300例数据，数据结构为(300,1,79,95,79)，其中300表示采样的个体，1代表通道数，每个单个样本的结构是(79,95,79)，其中79表示MRI数据采样的切片数，`95*79`代表每个切片的图像。
- 训练集的标注结果，样本的标签文件包含id和label字段，其中id表示图像序号，label表示图像对应的标签，0代表阿尔茨海默症样本，1代表轻度认知障碍样本，2代表正常样本。

In [1]:
import os
import h5py
import imageio
import pandas as pd
import numpy as np
from PIL import Image

In [3]:
train = h5py.File("./data/train/train_pre_data.h5")
df_labels = pd.read_csv("./data/train/train_pre_label.csv")

查看所有的主键

In [20]:
[key for key in train.keys()]

['data']

In [4]:
df_labels.head()

,id,label
0,0,1
1,1,0
2,2,0
3,3,1
4,4,0


In [7]:
pd.get_dummies(df_labels["label"], prefix="label")

,label_0,label_1,label_2
0,0,1,0
1,1,0,0
2,1,0,0
3,0,1,0
4,1,0,0
...,...,...,...
295,0,0,1
296,0,1,0
297,0,1,0
298,1,0,0


### 可视化探索

Python中常用的图像处理库为PIL，其中涉及的基本概念有:
- 通道(Brand):每张图片由一个或者多个通道组成，对于RGB图像由R、G、B三个通道构成，而对于灰度图像则只有一个通道，可以通过getbrands()方法来获取通道信息。
- 模式(Mode):图像的模式定义了图像的类型和像素的宽度，支持的模式有如下几种：
    - 1:1位像素，表示黑和白，但是存储的时候每个像素存储为8bit.
    - L:8位像素，表示黑和白.
    - P:8位像素，使用调色板映射到其他模式。
    - RGB:3x8位像素，为真彩色。
    - RGBA:4x8位像素，有透明通道的真彩色。
    - CMYK:4x8位像素，颜色分离。
    - YCbCr:3x8位像素，彩色视频格式。
    - I:32位整型像素。
    - F:32位浮点型像素。
- 尺寸(Size):图片的大小，二元数组，包含水平和垂直方向上的像素数。
- 坐标系()
- 调色板()
- 信息()
- 滤波器:将多个输入图像映射为一个输出像素的几何操作，支持4个不同采样滤波器：
    - NEAREST:最近滤波器，从输入图像中选取最近的像素作为输出像素，忽略其他的所有像素
    - BILINEAR:双线性滤波
    - BICUBIC:双立方滤波
    - ANTIALIAS:平滑滤波

In [42]:
# 创建temp目录存放一些临时处理的结果
dir_path = "./temp/"
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

In [43]:
idx = 0
sample = train["data"][idx, 0]
image_seq = 40
resize = 4
image_width, image_height = sample[image_seq].shape
sample_image = Image.fromarray(sample[image_seq]).convert("L")
#将图像放大4倍保存并查看
sample_image.resize((image_width * resize, image_height * resize), Image.ANTIALIAS).save(dir_path + "temp.jpg")
sample_image.show()

In [50]:
frames = []
for layer_img in one_sample:
    img = Image.fromarray(layer_img).convert('L')
    #将数据放大3倍
    img.resize((image_width * 3, image_height * 3),Image.ANTIALIAS).save(dir_path + 'temp.jpg')
    frames.append(imageio.imread(dir_path + 'temp.jpg'))
imageio.mimsave(dir_path +'{0}.gif'.format(idx), frames, 'GIF', duration = 0.1)